In [1]:
from random import randint

class GA:
    def __init__(self, param = None, problParam = None):
        self.__param = param
        self.__problParam = problParam
        self.__population = []
        
    @property
    def population(self):
        return self.__population
    
    def initialisation(self):
        for _ in range(0, self.__param['popSize']):
            c = Chromosome(self.__problParam)
            self.__population.append(c)
    
    def evaluation(self):
        for c in self.__population:
            c.fitness = self.__problParam['function'](c.repres)
            
    def bestChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness < best.fitness):
                best = c
        return best
        
    def worstChromosome(self):
        best = self.__population[0]
        for c in self.__population:
            if (c.fitness > best.fitness):
                best = c
        return best

    def selection(self):
        pos1 = randint(0, self.__param['popSize'] - 1)
        pos2 = randint(0, self.__param['popSize'] - 1)
        if (self.__population[pos1].fitness < self.__population[pos2].fitness):
            return pos1
        else:
            return pos2 
        
    
    def oneGeneration(self):
        newPop = []
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()

    def oneGenerationElitism(self):
        newPop = [self.bestChromosome()]
        for _ in range(self.__param['popSize'] - 1):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            newPop.append(off)
        self.__population = newPop
        self.evaluation()
        
    def oneGenerationSteadyState(self):
        for _ in range(self.__param['popSize']):
            p1 = self.__population[self.selection()]
            p2 = self.__population[self.selection()]
            off = p1.crossover(p2)
            off.mutation()
            off.fitness = self.__problParam['function'](off.repres)
            worst = self.worstChromosome()
            if (off.fitness < worst.fitness):
                worst = off

In [2]:
def generateARandomPermutation(n):
    perm = [i for i in range(1, n)]
    pos1 = randint(0, n - 2)
    pos2 = randint(0, n - 2)
    perm[pos1], perm[pos2] = perm[pos2], perm[pos1]
    return perm

def generateNewValue(lim1, lim2):
    return randint(lim1, lim2)

def binToInt(x):
    val = 0
    # x.reverse()
    for bit in x:
        val = val * 2 + bit
    return val

In [3]:
class Chromosome:
    def __init__(self, problParam = None):
        self.__problParam = problParam  #problParam has to store the number of nodes/cities
        self.__repres = generateARandomPermutation(self.__problParam['noNodes'])
        self.__fitness = 0.0
    
    @property
    def repres(self):
        return self.__repres 
    
    @property
    def fitness(self):
        return self.__fitness 
    
    @repres.setter
    def repres(self, l = []):
        self.__repres = l 
    
    @fitness.setter 
    def fitness(self, fit = 0.0):
        self.__fitness = fit 
    
    def crossover(self, c):
        # order XO
        pos1 = randint(-1, self.__problParam['noNodes'] - 1)
        pos2 = randint(-1, self.__problParam['noNodes'] - 1)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1 
        k = 0
        newrepres = self.__repres[pos1 : pos2]
        for el in c.__repres[pos2:] +c.__repres[:pos2]:
            if (el not in newrepres):
                if (len(newrepres) < self.__problParam['noNodes'] - pos1):
                    newrepres.append(el)
                else:
                    newrepres.insert(k, el)
                    k += 1

        offspring = Chromosome(self.__problParam)
        offspring.repres = newrepres
        return offspring
    
    def mutation(self):
        # insert mutation
        pos1 = randint(0, self.__problParam['noNodes'] - 2)
        pos2 = randint(0, self.__problParam['noNodes'] - 2)
        if (pos2 < pos1):
            pos1, pos2 = pos2, pos1
        el = self.__repres[pos2]
        del self.__repres[pos2]
        self.__repres.insert(pos1 + 1, el)
        
    def __str__(self):
        return "\nChromo: " + str(self.__repres) + " has fit: " + str(self.__fitness)
    
    def __repr__(self):
        return self.__str__()
    
    def __eq__(self, c):
        return self.__repres == c.__repres and self.__fitness == c.__fitness

In [4]:
def readFromFile(filename):
    nr_of_nodes = 0
    matrix_of_costs = []
    start = 0
    finish = 0
    with open(filename, "r") as reader:
        nr_of_nodes = int(reader.readline())
        for i in range(nr_of_nodes):
            nodes = reader.readline()
            nodes = nodes.split(",")
            nodes_value = []
            for n in nodes:
                nodes_value.append(int(n))
            matrix_of_costs.append(nodes_value)
    return [nr_of_nodes, matrix_of_costs]


def writeToFile(filenameOut, data):
    with open(filenameOut, "w+") as writer:
        writer.write(str(data[0]))
        writer.write("\n")
    data.pop(0)
    with open(filenameOut, "a") as writer:
        for line in data:
            writer.write(str(line))
            writer.write("\n")

In [5]:
def fitnessFunction(permutation, matrixOfCosts):
    lastNodeIndex = 0
    cost = 0
    for node in permutation:
        cost += matrixOfCosts[lastNodeIndex][node]
        lastNodeIndex = node
    cost += matrixOfCosts[lastNodeIndex][0]
    return cost
    

In [43]:
import random

nrOfNodes, matrixOfCosts = readFromFile("data.txt")
gaParam = {'popSize': 100, 'noGen': 100}
funct = lambda x: fitnessFunction(x, matrixOfCosts)
problParam = {"noNodes": nrOfNodes, "function": funct}

ga = GA(gaParam, problParam)
ga.initialisation()
ga.evaluation()

min = 100000000
path = []

for g in range(gaParam['noGen']):
#     logic alg
#     ga.oneGeneration()
    ga.oneGenerationElitism()
#    ga.oneGenerationSteadyState()

    bestChromo = ga.bestChromosome()
    print('Best solution in generation ' + str(g) + ' is: x = ' + str(bestChromo.repres) + ' f(x) = ' + str(bestChromo.fitness))
    if bestChromo.fitness < min:
        min = bestChromo.fitness
        path = bestChromo.repres
print("Best path: ", [0] + path)
print("With cost:", min)

Best solution in generation 0 is: x = [3, 2, 1, 5, 4] f(x) = 28
Best solution in generation 1 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 2 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 3 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 4 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 5 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 6 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 7 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 8 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 9 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 10 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 11 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 12 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 13 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 14 is: x = [2, 1, 3, 5, 4] f(x) = 27
Best solution in generation 15 is: 